# Imports

In [1]:
import pandas as pd 
import networkx as nx
import matplotlib.pyplot as plt

#  Taking raw network

In [2]:
# Get uci raw and convert it to uci.txt format (t, u, v)
cols = ['u','v','weight','timestamp']
df = pd.read_csv('rel.rating', sep=" ",  names=cols, skiprows=(3))
df

,u,v,weight,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
...,...,...,...,...
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795
99998,13,225,2,882399156


In [4]:
# Add "top_" prefix to "u" column and "bottom_" prefix to "v" column
df['u'] = df['u'].apply(lambda x: f"top_{x}")
df['v'] = df['v'].apply(lambda x: f"bottom_{x}")
df

,u,v,weight,timestamp
0,top_196,bottom_242,3,881250949
1,top_186,bottom_302,3,891717742
2,top_22,bottom_377,1,878887116
3,top_244,bottom_51,2,880606923
4,top_166,bottom_346,1,886397596
...,...,...,...,...
99995,top_880,bottom_476,3,880175444
99996,top_716,bottom_204,5,879795543
99997,top_276,bottom_1090,1,874795795
99998,top_13,bottom_225,2,882399156


In [5]:
df.drop(columns=["weight"], inplace=True)
df

,u,v,timestamp
0,top_196,bottom_242,881250949
1,top_186,bottom_302,891717742
2,top_22,bottom_377,878887116
3,top_244,bottom_51,880606923
4,top_166,bottom_346,886397596
...,...,...,...
99995,top_880,bottom_476,880175444
99996,top_716,bottom_204,879795543
99997,top_276,bottom_1090,874795795
99998,top_13,bottom_225,882399156


In [6]:
# Changing the position of the columns
df = df[['timestamp', 'u', 'v']]
df

,timestamp,u,v
0,881250949,top_196,bottom_242
1,891717742,top_186,bottom_302
2,878887116,top_22,bottom_377
3,880606923,top_244,bottom_51
4,886397596,top_166,bottom_346
...,...,...,...
99995,880175444,top_880,bottom_476
99996,879795543,top_716,bottom_204
99997,874795795,top_276,bottom_1090
99998,882399156,top_13,bottom_225


In [7]:
df['timestamp'] = df['timestamp'].astype(int)
df

,timestamp,u,v
0,881250949,top_196,bottom_242
1,891717742,top_186,bottom_302
2,878887116,top_22,bottom_377
3,880606923,top_244,bottom_51
4,886397596,top_166,bottom_346
...,...,...,...
99995,880175444,top_880,bottom_476
99996,879795543,top_716,bottom_204
99997,874795795,top_276,bottom_1090
99998,882399156,top_13,bottom_225


In [8]:
# Check how many occurances we have in every timestamp
timestamps_grouped = df.groupby('timestamp')[['u', 'v']].apply(lambda x: x.shape[0])
print(timestamps_grouped)

timestamp
874724710    1
874724727    1
874724754    1
874724781    1
874724843    1
            ..
893286550    3
893286584    1
893286603    1
893286637    3
893286638    7
Length: 49282, dtype: int64


In [10]:
df_time = pd.DataFrame()

In [11]:
# Convert timestamps to datetime format
df_time['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df_time

,timestamp
0,1997-12-04 15:55:49
1,1998-04-04 19:22:22
2,1997-11-07 07:18:36
3,1997-11-27 05:02:03
4,1998-02-02 05:33:16
...,...
99995,1997-11-22 05:10:44
99996,1997-11-17 19:39:03
99997,1997-09-20 22:49:55
99998,1997-12-17 22:52:36


In [12]:
# Calculate total time span
time_span = df_time['timestamp'].max() - df_time['timestamp'].min()
time_span

Timedelta('214 days 20:05:28')

In [13]:
# Convert time span to days
total_days = divmod(time_span.total_seconds(), 86400)[0]
print("Total number of days:", total_days)

Total number of days: 214.0


In [14]:
unique_timestamps = df_time['timestamp'].nunique()

print("Total different timestamps:", unique_timestamps)

Total different timestamps: 49282


# Remove direction then ordering by `t`, `u`, `v`, remove self-loops, duplicates, then save network

## Remove direction and sorting the link stream by `t`, `u`, `v`

In [15]:
# Swap 'u' and 'v' values where 'u' > 'v'
mask = df['u'] > df['v']
df.loc[mask, ['u', 'v']] = df.loc[mask, ['v', 'u']].values

# Sort DataFrame by timestamp, u, and v columns
df.sort_values(by=['timestamp', 'u', 'v'], inplace=True)

# Now df contains the sorted DataFrame with 'u' <= 'v'
df

,timestamp,u,v
214,874724710,bottom_255,top_259
83965,874724727,bottom_286,top_259
43027,874724754,bottom_298,top_259
21396,874724781,bottom_185,top_259
82655,874724843,bottom_173,top_259
...,...,...,...
73008,893286638,bottom_313,top_729
46574,893286638,bottom_328,top_729
52134,893286638,bottom_333,top_729
46773,893286638,bottom_689,top_729


In [17]:
# Sort DataFrame by timestamp, u, and v columns
df_sorted = df.sort_values(by=['timestamp', 'u', 'v'])

# Check if the DataFrame is sorted by 'u' and 'v' columns
if not df_sorted[['u', 'v']].apply(lambda x: x['v'] >= x['u'], axis=1).all():
    print("PROBLEM!!! Unsorted nodes in the DataFrame.")
else:
    print("DataFrame is sorted by 'u' and 'v'.")

# Check if the DataFrame is sorted by 'timestamp'
if not df_sorted['timestamp'].is_monotonic_increasing:
    print("PROBLEM!!! DataFrame is not sorted by 'timestamp'.")
else:
    print("DataFrame is sorted by 'timestamp'.")


DataFrame is sorted by 'u' and 'v'.
DataFrame is sorted by 'timestamp'.


## Removing self-loops

In [21]:
# Check for self-loops
self_loops = df['u'] == df['v']

if self_loops.any():
    # Print self-loops
    print("Self-loops:")
    print(df[self_loops])

    # Remove self-loops
    df_no_self_loops = df[~self_loops] # Not self-loops 
    
    # Display the DataFrame without self-loops
    print("\nDataFrame without self-loops:")
    print(df_no_self_loops)
    df = df_no_self_loops
else:
    print("No self-loops found.")

No self-loops found.


## Removing duplicate rows

In [22]:
# Count duplicate rows
duplicate_count = df.duplicated(keep=False).sum()

if duplicate_count > 0:
    # Print all duplicate rows
    print("Duplicate rows:")
    print(df[df.duplicated(keep=False)])
    
    # Remove duplicate rows
    df_no_duplicates = df.drop_duplicates()
    
    # Display the DataFrame without duplicates
    print("\nDataFrame without duplicates:")
    print(df_no_duplicates)
else:
    print("No duplicates found.")

No duplicates found.


In [24]:
#df = df_no_duplicates
df

,timestamp,u,v
214,874724710,bottom_255,top_259
83965,874724727,bottom_286,top_259
43027,874724754,bottom_298,top_259
21396,874724781,bottom_185,top_259
82655,874724843,bottom_173,top_259
...,...,...,...
73008,893286638,bottom_313,top_729
46574,893286638,bottom_328,top_729
52134,893286638,bottom_333,top_729
46773,893286638,bottom_689,top_729


In [25]:
# Sort DataFrame by timestamp, u, and v columns
df_sorted = df.sort_values(by=['timestamp', 'u', 'v'])

# Check if the DataFrame is sorted by 'u' and 'v' columns
if not df_sorted[['u', 'v']].apply(lambda x: x['v'] >= x['u'], axis=1).all():
    print("PROBLEM!!! Unsorted nodes in the DataFrame.")
else:
    print("DataFrame is sorted by 'u' and 'v'.")

# Check if the DataFrame is sorted by 'timestamp'
if not df_sorted['timestamp'].is_monotonic_increasing:
    print("PROBLEM!!! DataFrame is not sorted by 'timestamp'.")
else:
    print("DataFrame is sorted by 'timestamp'.")


DataFrame is sorted by 'u' and 'v'.
DataFrame is sorted by 'timestamp'.


## Saving the link stream

In [27]:
df.to_csv(r'movielens.txt', header=None, index=None, sep=' ')